In [ ]:
pip install llama-index langfuse openai nest_asyncio

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")
 

Langfuse client is authenticated and ready!


In [ ]:
pip install openinference-instrumentation-llama-index

In [3]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
 
# Initialize LlamaIndex instrumentation
LlamaIndexInstrumentor().instrument()

In [4]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-4o-mini")

Settings.embed_model = embed_model
Settings.llm = llm

In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_dir="data", recursive=True).load_data(num_workers=4)

documents

[Document(id_='0a584c83-3182-41ee-b65b-76b4fb7dc178', embedding=None, metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-llamaindex\\data\\Paul_Graham_(programmer).pdf', 'file_type': 'application/pdf', 'file_size': 284816, 'creation_date': '2025-11-28', 'last_modified_date': '1979-12-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Paul Graham\nGraham, c. 2004\nBorn November 13, 1964[1]\nWeymouth, Dorset, England[2]\nCitizenshipBritish\nAmerican\nEducation Cornell University (BA)\nHarvard University (MS, PhD)\nKnown for Viaweb\nY Combinator\nHacker News\nHackers & Pai

In [ ]:
from llama_index.core import VectorStoreIndex

# creating index without chunking 
index = VectorStoreIndex.from_documents(documents)

In [8]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=500, chunk_overlap=50),
        TitleExtractor(),
        embed_model
    ]
)

nodes = pipeline.run(documents=documents)
index = VectorStoreIndex(nodes)

100%|██████████| 6/6 [00:08<00:00,  1.38s/it]


In [8]:
retriever = index.as_retriever()
retriever

In [10]:
response = retriever.retrieve("tell me about paul graham in 3 sentences")
response

[NodeWithScore(node=TextNode(id_='7bb39fca-c19d-4dac-9bba-7e2cd4c934fe', embedding=None, metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-llamaindex\\data\\Paul_Graham_(programmer).pdf', 'file_type': 'application/pdf', 'file_size': 284816, 'creation_date': '2025-11-28', 'last_modified_date': '1979-12-31', 'document_title': '"Paul Graham: A Multifaceted Innovator in Computer Science, Entrepreneurship, and Philosophy"'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='dd463130-4388-46cc-80dd-234728834727', node_type='4', metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-ll

In [ ]:
# Retreive will only get documents from vectorstore index using similarity search
# Here Query Engine will do both Retrieve and also send it to LLM for summarization

query_engine = index.as_query_engine()
query_engine

In [18]:
response = query_engine.query("tell me about paul graham in 3 sentences")
response

Response(response='Paul Graham is an English-American computer scientist, entrepreneur, and writer, known for co-founding the startup accelerator Y Combinator and creating the programming language Arc. He has authored several influential books and essays, including "Hackers & Painters," and is recognized for his contributions to the tech community through platforms like Hacker News. Graham holds degrees in philosophy and computer science from Cornell and Harvard universities, respectively, and has maintained a permanent residence in England since 2016.', source_nodes=[NodeWithScore(node=TextNode(id_='7bb39fca-c19d-4dac-9bba-7e2cd4c934fe', embedding=None, metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-llamaindex\\data\\Paul_Graham_(programmer).pdf', 'file_type': 'application/pdf', 'file_size': 284816, 'creation_date': '2025-11-28', 'last_modified_date': '1979-12-31', 'document_title': '"Paul Graham: A Multifaceted I

In [13]:
from IPython.display import display, Markdown

display(Markdown(response.response))

Paul Graham is an English-American computer scientist, entrepreneur, and writer, known for co-founding the startup accelerator Y Combinator and creating the programming language Arc. He has authored several influential books and essays, including "Hackers & Painters," and is recognized for his contributions to the tech community through platforms like Hacker News. Born in England and raised in the United States, he holds degrees in philosophy and computer science from Cornell and Harvard universities, respectively.

Customising Retriever and Synthensizer Options

In [9]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer, Document

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=6
)

In [12]:
retrieved_nodes = retriever.retrieve("tell me about paul graham in 3 sentences")
retrieved_nodes

[NodeWithScore(node=TextNode(id_='196795d2-f833-4b50-92e8-7b479e687ab0', embedding=None, metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-llamaindex\\data\\Paul_Graham_(programmer).pdf', 'file_type': 'application/pdf', 'file_size': 284816, 'creation_date': '2025-11-28', 'last_modified_date': '1979-12-31', 'document_title': '"Paul Graham: A Multifaceted Journey Through Innovation, Academia, and Art"'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0a584c83-3182-41ee-b65b-76b4fb7dc178', node_type='4', metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-llamaindex\\data\\Pa

In [13]:
response_synthesizer = get_response_synthesizer()
response_synthesizer.synthesize("tell me about paul graham in 3 sentences", nodes=retrieved_nodes)

Response(response='Paul Graham is an English-American computer scientist, writer, entrepreneur, and investor, known for co-founding Y Combinator and creating the startup Viaweb. He holds degrees in philosophy from Cornell University and in computer science from Harvard University, and has authored several influential programming books and essays. Graham has also studied painting, reflecting his multifaceted interests in both technology and art.', source_nodes=[NodeWithScore(node=TextNode(id_='196795d2-f833-4b50-92e8-7b479e687ab0', embedding=None, metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-llamaindex\\data\\Paul_Graham_(programmer).pdf', 'file_type': 'application/pdf', 'file_size': 284816, 'creation_date': '2025-11-28', 'last_modified_date': '1979-12-31', 'document_title': '"Paul Graham: A Multifaceted Journey Through Innovation, Academia, and Art"'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file

In [14]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

query_engine.query("tell me about paul graham in 3 sentences")

Response(response='Paul Graham is an English-American computer scientist, writer, entrepreneur, and investor, known for co-founding Y Combinator and creating the startup Viaweb. He holds degrees in philosophy from Cornell University and in computer science from Harvard University, and he has authored several influential books and essays on programming and entrepreneurship. Graham has also studied painting, reflecting his multifaceted interests in both technology and art.', source_nodes=[NodeWithScore(node=TextNode(id_='196795d2-f833-4b50-92e8-7b479e687ab0', embedding=None, metadata={'page_label': '1', 'file_name': 'Paul_Graham_(programmer).pdf', 'file_path': 'c:\\Leela\\genai_workspace\\07-llamaindex\\data\\Paul_Graham_(programmer).pdf', 'file_type': 'application/pdf', 'file_size': 284816, 'creation_date': '2025-11-28', 'last_modified_date': '1979-12-31', 'document_title': '"Paul Graham: A Multifaceted Journey Through Innovation, Academia, and Art"'}, excluded_embed_metadata_keys=['fil

Node Post Processor Example

In [ ]:
pip install spacy

In [5]:
from llama_index.core import Document
from llama_index.core import VectorStoreIndex

documents = [
    Document(text="The quick brown for jumps over the lazy dog"),
    Document(text="AI and Machine Learning are tranforming industries"),
    Document(text="The stock market saw a significant rise in AI stocks"),
    Document(text="Python programming is fun and highly versatile"),
    Document(text="The quick development of AI tools like LLamaIndex simplifies many workflows")
]

index = VectorStoreIndex.from_documents(documents=documents)

In [6]:
from llama_index.core.postprocessor import KeywordNodePostprocessor

node_post_processor = KeywordNodePostprocessor(
    exclude_keywords=["quick"],
    required_keywords=["AI"]
)


In [7]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3
)

query_engine = RetrieverQueryEngine(
    retriever=retriever, node_postprocessors=[node_post_processor]
)

In [8]:
query_engine.query("Tell me about AI and also python.")

Response(response='AI, or Artificial Intelligence, involves the development of systems that can perform tasks typically requiring human intelligence, such as understanding natural language, recognizing patterns, and making decisions. Machine Learning, a subset of AI, focuses on algorithms that allow computers to learn from and make predictions based on data.\n\nPython is a popular programming language widely used in AI and Machine Learning due to its simplicity and the availability of powerful libraries and frameworks, such as TensorFlow and PyTorch. These tools facilitate the development of AI models and applications, making Python a preferred choice for many developers in the field.', source_nodes=[NodeWithScore(node=TextNode(id_='73db0d0e-660c-4793-9fd8-a77a1e5ce143', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='023ced57-d2ae-492c-87fb-11a1c3c4e6f8', node_type='4',

In [9]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="data2").load_data(num_workers=4)

index = VectorStoreIndex.from_documents(documents=documents)

query_engine = index.as_query_engine()

In [11]:
response = query_engine.query("Give me director names and count of episodes as json")
print(response.response)

```json
{
    "Gregg Kavet": 2,
    "Andy Robin": 2,
    "Andy Ackerman": 20,
    "David Mandel": 3,
    "Peter Mehlman": 3,
    "Jennifer Crittenden": 3,
    "Dan O'Keefe": 3,
    "Alec Berg": 3,
    "Spike Feresten": 3,
    "Bruce Eric Kaplan": 2,
    "Steve Koren": 2,
    "Steve Lookner": 1,
    "Larry David": 2,
    "Larry Charles": 4,
    "Tom Cherones": 12,
    "Elaine Pope": 1,
    "Joshua White": 1,
    "Don McEnery": 1,
    "Bill Masters": 1
}
```


In [12]:
response2 = query_engine.query("Give me only top 5 based on count")
print(response2)

1. Larry David - 12 episodes
2. Tom Cherones - 12 episodes
3. Larry Charles - 10 episodes
4. Peter Mehlman - 6 episodes
5. Bruce Kirschbaum - 4 episodes


Query Engine Doesn't Have Memory,we have to use Chat Engine for that

In [15]:
chat_engine = index.as_chat_engine()

In [16]:
response = chat_engine.chat("Give me director names and count of episodes as json")
print(response)

Based on the provided episode information, here is the JSON representation of director names along with the count of episodes they directed:

```json
{
  "directors": {
    "Larry David": 6,
    "Jerry Seinfeld": 6,
    "Art Wolff": 1,
    "Tom Cherones": 12,
    "Matt Goldman": 3,
    "Larry Charles": 4,
    "Peter Mehlman": 3,
    "Elaine Pope": 1,
    "David Steinberg": 2,
    "Joshua White": 1,
    "Don McEnery": 1,
    "Gregg Kavet": 3,
    "Andy Robin": 3,
    "Andy Ackerman": 12,
    "David Mandel": 3,
    "Peter Mehlman": 1,
    "Alec Berg": 3,
    "Spike Feresten": 3,
    "Bruce Eric Kaplan": 2,
    "Steve Koren": 2,
    "Dan O'Keefe": 3,
    "Jennifer Crittenden": 3,
    "Steve Lookner": 1
  }
}
```

This JSON object lists each director along with the total number of episodes they directed. If you have any other questions or need further details, feel free to ask!


In [17]:
response2 = chat_engine.chat("Give me only top 5 based on count")
print(response2)

Here is the JSON representation of the top 5 directors based on the count of episodes they directed:

```json
{
  "top_directors": {
    "Tom Cherones": 12,
    "Andy Ackerman": 12,
    "Larry David": 6,
    "Jerry Seinfeld": 6,
    "Larry Charles": 4
  }
}
```

This lists the top 5 directors along with the number of episodes they directed. If you need more information or have any other questions, just let me know!
